# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [142]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
#from sklearn.grid_search import GridSearchCV

from sklearn.model_selection import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

np.random.seed(50)

Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [143]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = len(newsgroups_test.target)
test_data, test_labels = newsgroups_test.data[int(num_test/2):], newsgroups_test.target[int(num_test/2):]
dev_data, dev_labels = newsgroups_test.data[:int(num_test/2)], newsgroups_test.target[:int(num_test/2)]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print('training label shape:', train_labels.shape)
print('test label shape:', test_labels.shape)
print('dev label shape:', dev_labels.shape)
print('labels names:', newsgroups_train.target_names)

training label shape: (2034,)
test label shape: (677,)
dev label shape: (676,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


(1) For each of the first 5 training examples, print the text of the message along with the label.

In [144]:
def P1(num_examples=5):
    for i in range (num_examples):
        
        print ("This is an example of Group:", train_labels[i], \
               "type: ",newsgroups_train.target_names[train_labels[i]])
        print ("_" * 80)
        print (train_data[i])
        print ("_" * 80)
        print ("\n")
        
        #print (train_labels[i])
P1()

This is an example of Group: 1 type:  comp.graphics
________________________________________________________________________________
Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
________________________________________________________________________________


This is an example of Group: 3 type:  talk.religion.misc
________________________________________________________________________________


Seems to be, barring evidence to th

(2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

In [145]:
def P2():
    vectorizer = CountVectorizer()
    feature_vec = vectorizer.fit_transform(train_data)
    
    print ("\n")
    #print (np.shape(feature_vec[1]))
    
    #Question a. 
    #Size of vocabulary
    print ("-"*80)
    print ("Question 2.a")
    print ("-"*80)
    
    print ("Shape of feature vector: ", np.shape(feature_vec))
    print ("Size of vocabulary: ", np.shape(feature_vec)[1])
    
    #Average size of non-zero feature values
    print ("Number of non-zero features in matrix: ", feature_vec.nnz)
    print ("Size of feature vector matrix: ", np.prod(np.shape(feature_vec)))
    print ("Fraction of non-zero entries in matrix: {0:0.5f}".format(feature_vec.nnz/(np.prod(np.shape(feature_vec)))))
    print ("Average size of non-zero feature values per document: {0:0.2f}"
           .format(feature_vec.nnz/(np.shape(feature_vec))[0]))
    print ("-"*80)
    print ("\n")
    
    #Question b
    print ("-"*80)
    print ("Question 2.b")
    print ("-"*80)
    
    #First and last feature names
    print ("First feature name:", vectorizer.get_feature_names()[0])
    print ("Last feature name:", vectorizer.get_feature_names()[len(vectorizer.get_feature_names()) - 1])
    print ("\n")
    
    #Question c
    print ("-"*80)
    print ("Question 2.c")
    print ("-"*80)
    
    #Specifying own vocabulary with [atheism", "graphics", "space", "religion"]
    my_vocab = ["atheism", "graphics", "space", "religion"]
    my_vectorizer = CountVectorizer(vocabulary=my_vocab)
    my_feature_vec = my_vectorizer.fit_transform(train_data)
    
    print ("Shape of my feature vector: ", np.shape(my_feature_vec))
    print ("Size of my vocabulary: ", np.shape(my_feature_vec)[1])
    
    #Average size of non-zero feature values
    print ("Number of non-zero features in matrix with my vocab: ", my_feature_vec.nnz)
    print ("Size of feature vector matrix with my vocab: ", np.prod(np.shape(my_feature_vec)))
    print ("Fraction of non-zero entries in matrix: {0:0.5f} with my vocab".format(my_feature_vec.nnz/(np.prod(np.shape(my_feature_vec)))))
    print ("Average size of non-zero feature values per document: {0:0.2f} with my vocab"
           .format(my_feature_vec.nnz/(np.shape(my_feature_vec))[0]))
    print ("-"*80)
    print ("\n")
    
    #Question d
    print ("-"*80)
    print ("Question 2.d")
    print ("-"*80)
    
    #Bi-gram vectorizer
    bigram_vectorizer = CountVectorizer(ngram_range=(1, 2),
                                        token_pattern=r'\b\w+\b', 
                                        min_df=1)
    bigram_feature_vec = bigram_vectorizer.fit_transform(train_data)
    print ("Size of bi-gram vocabulary: ", np.shape(bigram_feature_vec)[1])
    
    #Trigram vectorizer
    trigram_vectorizer = CountVectorizer(ngram_range=(1, 3),
                                        token_pattern=r'\b\w+\b', 
                                        min_df=1)
    trigram_feature_vec = trigram_vectorizer.fit_transform(train_data)
    print ("Size of tri-gram vocabulary: ", np.shape(trigram_feature_vec)[1])
    print ("\n")
    
    #Question e
    print ("-"*80)
    print ("Question 2.e")
    print ("-"*80)
    
    #Using mind_df of 10, build unigram vectorizer
    uni_min_df10_vectorizer = CountVectorizer(min_df = 10)
    uni_min_df10_feature_vec = uni_min_df10_vectorizer.fit_transform(train_data)
    print ("Size of unigram vocabulary with df = 10: ", np.shape(uni_min_df10_feature_vec)[1])
    print ("\n")
    
    #Question f
    print ("-"*80)
    print ("Question 2.f")
    print ("-"*80)
    
    #Get vocab for train data using original vectorizer one more time
    train_data_vectorizer = CountVectorizer()
    train_feature_vec = train_data_vectorizer.fit_transform(train_data)
    print ("Size of train data vocabulary: ", np.shape(train_feature_vec)[1])
    
    #Get vocab for dev data using original vectorizer
    dev_data_vectorizer = CountVectorizer()
    dev_feature_vec = dev_data_vectorizer.fit_transform(dev_data)
    print ("Size of dev data vocabulary: ", np.shape(dev_feature_vec)[1])
    
    #find out which keys are in dev data, but not in the training data
    #missing_key_cnt = 0
    #for key in dev_feature_vec : 
    #   if key not in train_feature_vec :
    #       missing_key_cnt += 1
   
    missing_keys = set(dev_data_vectorizer.vocabulary_.keys()) - set(train_data_vectorizer.vocabulary_.keys())
    missing_key_cnt = len(missing_keys)
    print ("Number of missing keys", missing_key_cnt)
            
    print ("Fraction of words from dev data missing in train data vocabulary: {0:.2f}"
           .format(missing_key_cnt/np.shape(dev_feature_vec)[1]))
    print ("\n")

    
    
P2()



--------------------------------------------------------------------------------
Question 2.a
--------------------------------------------------------------------------------
Shape of feature vector:  (2034, 26879)
Size of vocabulary:  26879
Number of non-zero features in matrix:  196700
Size of feature vector matrix:  54671886
Fraction of non-zero entries in matrix: 0.00360
Average size of non-zero feature values per document: 96.71
--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
Question 2.b
--------------------------------------------------------------------------------
First feature name: 00
Last feature name: zyxel


--------------------------------------------------------------------------------
Question 2.c
--------------------------------------------------------------------------------
Shape of my feature vector:  (2034, 4)
Size of my vocabulary:  4
Number of no

(3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

a. Why doesn't nearest neighbors work well for this problem?

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

In [150]:
model_lr = LogisticRegression()
model_lr

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [158]:
def P3():
    
    #Unigram CountVectorizer
    vec = CountVectorizer()
    #create training vector
    X_train = vec.fit_transform(train_data)
    Y_train = train_labels
    
    #print shapes
    print ("Shape of training data", np.shape(X_train))
    print ("Shape of training labels", np.shape(Y_train))
    
    #Now let's try out different algorithms with this training set
    #kNN - find optimal value of k using GridSearch
    k_list = [2*k+3 for k in range (25)]
    k_values = {'n_neighbors' : k_list}
    model_knn = KNeighborsClassifier()
    clf_knn = GridSearchCV(model_knn, k_values, cv = 5)
    clf_knn.fit(X_train, Y_train)
    print ("-"*80)
    print ("Trying out Grid Search on k values for KNN Algorithm")
    print ("")
    print(clf_knn.cv_results_['mean_test_score'])
    print("The best k value for KNN:", clf_knn.best_params_)
    print("The best f1 score for KNN", clf_knn.best_score_)
    print ("-"*80)
    print ("\n")

    #Now Multinomial NB
    model_nb = MultinomialNB()
    alpha_values = {'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 1.05, 1.1, 5, 10]}
    clf_nb = GridSearchCV(model_nb, alpha_values, cv =5)
    clf_nb.fit(X_train, Y_train)
    print ("-"*80)
    print ("Trying out Grid Search on alpha values for Multinomial Naive Bayes Algorithm")
    print(clf_nb.cv_results_['mean_test_score'])
    print("The best alpha value for Multinomial NB", clf_nb.best_params_)
    print("The best f1 score for Multinomial NB", clf_nb.best_score_)
    print ("-"*80)
    print ("\n")
    
    #Logistic Regression
    model_lr = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=500)
    C_values = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
    clf_lr = GridSearchCV(model_lr, C_values, cv =5)
    clf_lr.fit(X_train, Y_train)
    print ("-"*80)
    print ("Trying out Grid Search on C values for Logistic Regression")
    print(clf_lr.cv_results_['mean_test_score'])
    print("The best alpha value for Logistic Regression", clf_lr.best_params_)
    print("The best f1 score for Logistic Regression", clf_lr.best_score_)
    print ("-"*80)
    print ("\n")
    
P3()

Shape of training data (2034, 26879)
Shape of training labels (2034,)
--------------------------------------------------------------------------------
Trying out Grid Search on k values for KNN Algorithm

[0.40363815 0.41494592 0.43608653 0.42822026 0.4346116  0.43952802
 0.43215339 0.42772861 0.44346116 0.44001967 0.42969518 0.43854474
 0.43510324 0.43313668 0.43608653 0.4252704  0.42428712 0.42084562
 0.42625369 0.42281219 0.43117011 0.42576205 0.42772861 0.43166175
 0.43756146]
The best k value for KNN: {'n_neighbors': 19}
The best f1 score for KNN 0.44346116027531957
--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
Trying out Grid Search on alpha values for Multinomial Naive Bayes Algorithm
[0.82497542 0.8259587  0.82546706 0.82841691 0.82055064 0.79990167
 0.79842675 0.7979351  0.75122911 0.71435595]
The best alpha value for Multinomial NB {'alpha': 0.01}
The best f1 

ANSWER:

(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

In [10]:
#def P4():
### STUDENT START ###

### STUDENT END ###
#P4()

ANSWER:

(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

In [11]:
#def empty_preprocessor(s):
#    return s

#def better_preprocessor(s):
### STUDENT START ###

### STUDENT END ###

#def P5():
### STUDENT START ###

### STUDENT END ###
#P5()

(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

In [12]:
#def P6():
    # Keep this random seed here to make comparison easier.
    #np.random.seed(0)

    ### STUDENT START ###
    
    ### STUDENT END ###
#P6()

(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

In [13]:
#def P7():
### STUDENT START ###

## STUDENT END ###
#P7()

ANSWER:

(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.